In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import pandas as pd
import os
import random
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [3]:
#task 2
def time(x):
    total = 0
    minutes = re.findall('([0-9]+) mins*',x)
    hours = re.findall('([0-9]+) hour*',x)
    half_hours = re.findall('1 2 hour*', x)
    if len(minutes) != 0:
        count = sum([int(i) for i in minutes])
        total = total + count
    if len(hours) != 0:
        count = sum([int(i) * 60 for i in hours])
        total = total + (count)
    if len(half_hours) != 0:
        for i in half_hours:
            total = total - 30
    return total

In [54]:
fp = os.path.join("Downloads/assignment1","trainRecipes.json.gz")
data = []
for d in readGz(fp):
    data.append(d)

trainrecipes = pd.DataFrame(data)
trainrecipes['steps'] = trainrecipes['steps'].apply(lambda x: re.sub("[^0-9a-zA-Z]+"," ", x).lower())
trainrecipes['len'] = trainrecipes['steps'].apply(lambda x: len(x))
trainrecipes['num_ingred'] = trainrecipes['ingredients'].apply(lambda x: len(x))
trainrecipes['nums_explicit'] = trainrecipes['steps'].apply(lambda x: time(x))
trainrecipes['len_name'] = trainrecipes['name'].apply(lambda x: len(x))
trainrecipes['steps'] = trainrecipes['steps'].apply(lambda x: re.sub('[0-9]', '',x))
trainrecipes['oven'] = trainrecipes['steps'].apply(lambda x: 1 if 'oven' in x else 0)
trainrecipes['refrigerator'] = trainrecipes['steps'].apply(lambda x: 1 if 'refrigerator' in x else 0)
trainrecipes['overnight'] = trainrecipes['steps'].apply(lambda x: 1 if 'overnight' in x else 0)
trainrecipes['cook'] = trainrecipes['steps'].apply(lambda x: 1 if 'cook' in x else 0)
trainrecipes['bake'] = trainrecipes['steps'].apply(lambda x: 1 if 'bake' in x else 0)
trainrecipes['cool'] = trainrecipes['steps'].apply(lambda x: 1 if 'cool' in x else 0)
trainrecipes

,name,minutes,contributor_id,submitted,steps,description,ingredients,recipe_id,len,num_ingred,nums_explicit,len_name,oven,refrigerator,overnight,cook,bake,cool
0,sexy fried eggs for sunday brunch,10,14298494,2004-05-21,heat a ridged griddle pan lightly brush the to...,"this is from silvana franco's book ""family"" wh...","[plum tomato, ciabatta, olive oil, egg, garlic...",06432987,730,9,5,33,0,0,0,1,0,0
1,double delicious cookie bars,40,26865936,2007-08-27,preheat oven to f in x inch baking pan melt bu...,"from ""all time favorite recipes"". for fun, try...","[butter, graham cracker crumbs, sweetened cond...",98015212,272,5,30,28,1,0,0,0,1,1
2,gorgeous chocolate muffins,50,11206582,2008-05-28,in a bowl cream butter sugar vanilla sugar and...,this is a true treat for all chocolate lovers!...,"[butter, sugar, vanilla sugar, egg, milk, vine...",27596743,400,11,30,26,1,0,0,0,1,0
3,amy s quick kielbasa skillet dinner,25,53076165,2003-10-03,combine frozen potatoes and sliced kielbasa in...,i made this up one night when i needed a quick...,"[frozen southern style hash brown potatoes, ki...",74041916,216,4,0,35,0,0,0,1,0,0
4,old fashioned chocolate bread pudding with kah...,55,56214328,2008-10-08,preheat oven to f begin by buttering small ra...,are you tired of serving the 'same old - same ...,"[butter, granulated sugar, whole milk, heavy c...",52053290,906,18,30,63,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,green chile enchiladas,70,51649462,2002-12-19,cook beef and garlic powder in skillet stirrin...,quick and easy ground beef dinner.,"[lean ground beef, garlic powder, colby-monter...",62771271,667,11,40,22,0,0,0,1,1,0
199996,leftover ham with pasta and broccoli casserole,37,97438710,2007-04-10,preheat oven to degrees cook pasta according ...,great use for leftover easter ham.,"[spiral shaped pasta, cubed ham, frozen brocco...",70012752,712,15,13,46,1,0,0,1,1,0
199997,oatmeal cookies with raisins and cranberries,70,71977973,2008-02-10,heat oven to f beat together butter and sugars...,i adapted this recipe for my husband who adore...,"[butter, brown sugar, granulated sugar, eggs, ...",45070289,462,13,48,44,1,0,0,1,1,1
199998,polenta pizza with brie and red grapes,195,85742694,2009-02-11,combine yeast with cup warm water in medium ...,i positively love this recipe! this is for whe...,"[yeast, warm water, olive oil, kosher salt, al...",93801897,860,11,187,38,1,0,0,0,1,1


In [55]:
CountVec = CountVectorizer(ngram_range=(1,1), tokenizer = lambda x: x.split(), max_features = 2000, stop_words ='english')
Count_data = CountVec.fit_transform(trainrecipes['steps'])
cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
training = pd.concat([trainrecipes[['len','num_ingred','nums_explicit','len_name','oven','refrigerator','overnight','cook','bake','cool']],cv_dataframe], axis = 1)
training

,len,num_ingred,nums_explicit,len_name,oven,refrigerator,overnight,cook,bake,cool,...,yogurt,yolk,yolks,yum,yummy,zest,zip,ziploc,ziplock,zucchini
0,730,9,5,33,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,272,5,30,28,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,400,11,30,26,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,216,4,0,35,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,906,18,30,63,1,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,667,11,40,22,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
199996,712,15,13,46,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
199997,462,13,48,44,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
199998,860,11,187,38,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [56]:
clf = Ridge(alpha=1.5)
clf.fit(training, trainrecipes['minutes'])


Ridge(alpha=1.5)

In [57]:
ids = []
steps = []
ingredients = []
names = []
for d in readGz(os.path.join("downloads/assignment1","testRecipes.json.gz")):
    ids.append(d['recipe_id'])
    steps.append(d['steps'])
    ingredients.append(d['ingredients'])
    names.append(d['name'])

test = pd.DataFrame()
test['name'] = names
test['recipe_id'] = ids
test['steps'] = steps
test['ingredients'] = ingredients
test['steps'] = test['steps'].apply(lambda x: re.sub("[^0-9a-zA-Z]+"," ", x).lower())
test['len'] = test['steps'].apply(lambda x: len(x))
test['num_ingred'] = test['ingredients'].apply(lambda x: len(x))
test['nums_explicit'] = test['steps'].apply(lambda x: time(x))
test['len_name'] = test['name'].apply(lambda x: len(x))
test['steps'] = test['steps'].apply(lambda x: re.sub('[0-9]', '',x))
test['oven'] = test['steps'].apply(lambda x: 1 if 'oven' in x else 0)
test['refrigerator'] = test['steps'].apply(lambda x: 1 if 'refrigerator' in x else 0)
test['overnight'] = test['steps'].apply(lambda x: 1 if 'overnight' in x else 0)
test['cook'] = test['steps'].apply(lambda x: 1 if 'cook' in x else 0)
test['bake'] = test['steps'].apply(lambda x: 1 if 'bake' in x else 0)
test['cool'] = test['steps'].apply(lambda x: 1 if 'cool' in x else 0)
test

,name,recipe_id,steps,ingredients,len,num_ingred,nums_explicit,len_name,oven,refrigerator,overnight,cook,bake,cool
0,rhubarb pie ii,79578623,combine all ingredients except butter or marga...,"[rhubarb, flour, sugar, salt, orange rind, but...",245,6,50,14,0,0,0,0,1,0
1,ww 6 points mediterranean roast chicken,34742403,preheat oven to f spray roasting rack with non...,"[thyme, dried oregano, olive oil, garlic clove...",669,10,65,39,1,0,0,0,0,0
2,sweet salty grilled cheese sandwich,27205316,spread a thin coat of butter or mayonnaise on ...,"[bread, sharp cheddar cheese, dates, butter]",417,4,6,35,0,0,0,1,0,0
3,taco pie in pie crust,15313464,brown meat drain fat and season unroll crust p...,"[refrigerated pie crust, ground beef, refried ...",387,13,40,21,1,0,0,0,1,0
4,blueberry bran muffins,72546246,preheat oven to f and grease a cup muffin pan...,"[raisin bran cereal, sugar, all-purpose flour,...",363,9,25,22,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,berry good spinach salad,63844579,place spinach in a bowl and then place the str...,"[baby spinach leaves, strawberries, basil leav...",296,5,0,24,0,0,0,0,0,0
9996,soul pot roast,62754221,preheat oven to degrees f place meat in a qu...,"[chuck roast, carrots, potatoes, yellow onions...",258,8,300,14,1,0,0,1,0,0
9997,bunny in the hole sandwich,83455838,preheat oven to f cut a hole in the centre of ...,"[eggs, whole wheat bread, cheddar cheese, gree...",610,7,15,26,1,0,0,1,1,0
9998,strawberry roll cake,84699987,preheat oven to degrees and line a rimmed coo...,"[flour, baking powder, salt, eggs, sugar, wate...",821,10,15,20,1,1,0,1,1,1


In [58]:
test_count = CountVec.transform(test['steps'])
test_cv =pd.DataFrame(test_count.toarray(),columns=CountVec.get_feature_names())
testing = pd.concat([test[['len','num_ingred','nums_explicit','len_name','oven','refrigerator','overnight','cook','bake','cool']],test_cv], axis = 1)
testing

,len,num_ingred,nums_explicit,len_name,oven,refrigerator,overnight,cook,bake,cool,...,yogurt,yolk,yolks,yum,yummy,zest,zip,ziploc,ziplock,zucchini
0,245,6,50,14,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,669,10,65,39,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,417,4,6,35,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,387,13,40,21,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,363,9,25,22,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,296,5,0,24,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,258,8,300,14,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,610,7,15,26,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
9998,821,10,15,20,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [59]:
x_pred =clf.predict(testing)
x_pred

array([33.28249943, 83.71154614, 22.4417951 , ..., 37.70298753,
       46.94671452, 28.22183868])

In [60]:
res = pd.DataFrame()
res['recipe_id'] = test['recipe_id']
res['pred'] = x_pred
res

,recipe_id,pred
0,79578623,33.282499
1,34742403,83.711546
2,27205316,22.441795
3,15313464,65.566400
4,72546246,25.552757
...,...,...
9995,63844579,10.566613
9996,62754221,153.631707
9997,83455838,37.702988
9998,84699987,46.946715


In [61]:
preds = open("prediction_Minutes.txt", 'w')
preds.write("recipe_id,prediction\n")
for i in range(0, res.shape[0]):
    preds.write(res['recipe_id'][i] + ',' + str(res['pred'][i]) + '\n')
preds.close()